# Tables 1, 2 (Proxies of Temperature)

Builds the portions of Tables 1, 2 that include W, Z

Requires that `run_exp_temp_proxies.py` is run

In [1]:
RPATH = '../results'
FPATH = '../figs'

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pickle as pkl

In [3]:
results = pd.read_csv(f"{RPATH}/all_res_test_anchor_prox_TempC_lamb_fixed.csv")
results = results.drop('Unnamed: 0', axis=1)

residuals = results.drop("Lambda", axis=1).query('Environment == "Test"')
lambs = results.drop("Residual", axis=1).query('Environment == "Test"')

mse = lambda v: np.mean(v**2)

# Get RMSE by season, city
pt = pd.pivot_table(residuals, 
                    index=['Test_Season', 'City'], 
                    columns = 'Estimator', 
                    aggfunc={'Residual': mse})

pt.columns = pt.columns.droplevel(0)
pt = pt.rename(columns = {
    'TAR (W)': 'PTAR (W)',
    'xTAR (W, Z)': 'xPTAR (W, Z)',
    'AR (W)': 'PAR (W)',
    'xAR (W, Z)': 'xPAR (W, Z)'
})
newcols = [
 'OLS',
 'PAR (W)',
 'xPAR (W, Z)',
 'PTAR (W)',
 'xPTAR (W, Z)', 
]
pt = pt[newcols].reset_index()

# Table 1 (W, Z)

In [4]:
wins = lambda v: int(np.sum(v < 0))
loss = lambda v: int(np.sum(v > 0))

lambs_ar = lambs.query(f'Environment == "Test" & Estimator == "AR (W)"').groupby(
    ['City', 'Test_Season']).mean()[['Lambda']]

lambs_ar = lambs_ar.reset_index().set_index(['City', 'Test_Season'])

pt_diff = pt.copy()
for est in newcols:
    pt_diff[est] = pt[est] - pt['OLS']

pt_pos_lamb = pt.set_index(['City', 'Test_Season']).merge(lambs_ar, left_index=True, right_index=True)
pt_pos_lamb = pt_pos_lamb.query("Lambda > 0").drop("Lambda", axis=1).reset_index()

pt_diff_pos_lamb = pt_pos_lamb.copy()
for est in newcols:
    pt_diff_pos_lamb[est] = pt_pos_lamb[est] - pt_pos_lamb['OLS']

lt = pd.melt(pt_pos_lamb, id_vars=['Test_Season', 'City'], value_name = 'MSE', var_name = 'Estimator')

mean_result = lt.groupby('Estimator', as_index=True).agg(
    **{'Mean': pd.NamedAgg(column='MSE', aggfunc=np.mean)}
).reindex(newcols)

pt_diff_long = pd.melt(pt_diff_pos_lamb, id_vars=['Test_Season', 'City'], value_name = 'MSE', var_name='Estimator')

diff_ols = pt_diff_long.groupby('Estimator', as_index=True).agg(
    **{'min': pd.NamedAgg(column='MSE', aggfunc=np.min), 
       'max': pd.NamedAgg(column='MSE', aggfunc=np.max)}
).reindex(newcols)

win_loss_ols = pt_diff_long.groupby('Estimator', as_index=True).agg(
    **{'Win': pd.NamedAgg(column='MSE', aggfunc=wins)}
).reindex(newcols)

print(pd.concat([mean_result, win_loss_ols.astype(int), diff_ols], axis=1).to_latex(float_format="{:.3f}".format))

\begin{tabular}{lrrrr}
\toprule
{} &  Mean &  Win &    min &   max \\
Estimator    &       &      &        &       \\
\midrule
OLS          & 0.537 &    0 &  0.000 & 0.000 \\
PAR (W)      & 0.531 &    6 & -0.037 & 0.006 \\
xPAR (W, Z)  & 0.531 &    6 & -0.039 & 0.007 \\
PTAR (W)     & 0.529 &    8 & -0.038 & 0.001 \\
xPTAR (W, Z) & 0.526 &    7 & -0.059 & 0.001 \\
\bottomrule
\end{tabular}



# Table 2 (W, Z)

In [5]:
pt_diff = pt.copy()
for est in newcols:
    pt_diff[est] = pt[est] - pt['OLS']

lt = pd.melt(pt, id_vars=['Test_Season', 'City'], value_name = 'MSE')

mean_result = lt.groupby('Estimator', as_index=True).agg(
    **{'Mean': pd.NamedAgg(column='MSE', aggfunc=np.mean)}
).reindex(newcols)

pt_diff_long = pd.melt(pt_diff, id_vars=['Test_Season', 'City'], value_name = 'MSE')

diff_ols = pt_diff_long.groupby('Estimator', as_index=True).agg(
    **{'Diff': pd.NamedAgg(column='MSE', aggfunc=np.mean),
       'min': pd.NamedAgg(column='MSE', aggfunc=np.min), 
       'max': pd.NamedAgg(column='MSE', aggfunc=np.max)}
).reindex(newcols)

print(pd.concat([mean_result, diff_ols], axis=1).to_latex(float_format="{:.3f}".format))

\begin{tabular}{lrrrr}
\toprule
{} &  Mean &   Diff &    min &   max \\
Estimator    &       &        &        &       \\
\midrule
OLS          & 0.457 &  0.000 &  0.000 & 0.000 \\
PAR (W)      & 0.454 & -0.002 & -0.037 & 0.006 \\
xPAR (W, Z)  & 0.454 & -0.003 & -0.039 & 0.007 \\
PTAR (W)     & 0.452 & -0.005 & -0.038 & 0.001 \\
xPTAR (W, Z) & 0.450 & -0.007 & -0.059 & 0.003 \\
\bottomrule
\end{tabular}

